# test-api.ipynb

Test API and helper functions

In [ ]:
import os, sys
from typing import Optional, List, Dict
sys.path.append(os.path.join(".."))
import hjson
import docstring_parser
import inspect
from llms_wrapper.llms import LLMS
from llms_wrapper.config import update_llm_config

In [ ]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        dict(llm="openai/o1"),        # restricted
        dict(llm="openai/o1-mini"),   # restricted
        dict(llm="openai/gpt-4.5-preview"),   # restricted?
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
        dict(
            llm="gemini/somemodel",
            max_input_tokens=100000,
            cost_per_prompt_token=0.0002,
            temperature=0,
        ), 
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
_ = update_llm_config(config)

In [ ]:
llms = LLMS(config,  use_phoenix=("http://0.0.0.0:6006/v1/traces", "llms_wrapper_test"))

In [ ]:
llms.list_aliases()

In [ ]:
# llms.known_models()

In [ ]:
llms.cost_per_token("openai/gpt-4o")

In [ ]:
llms.max_input_tokens("openai/gpt-4o")

In [ ]:
llms.max_output_tokens("openai/gpt-4o")

In [ ]:
llms["gemini/somemodel"].config

In [ ]:
llms.cost_per_token("gemini/gemini-1.5-flash")

In [ ]:
llms.cost_per_token("gemini/somemodel")

In [ ]:
llms.max_input_tokens("gemini/somemodel")

In [ ]:
llms.max_output_tokens("gemini/somemodel")

In [ ]:
msg1 = llms.make_messages("What is a monoid?")

In [ ]:
ret1 = llms.query(llmalias="openai/gpt-4.5-preview", return_cost=True, return_response=True, messages=msg1)
ret1

In [ ]:
msg2 = llms.make_messages("What is a monoid? Return a JSON dict that has the single key 'answer' that contains your answer.")

In [ ]:
ret2=llms.query(
    llmalias="openai/gpt-4o", 
    return_response=True,
    return_cost=True,
    messages=msg2, response_format=dict(type="json_object"))
ret2

In [ ]:
# ret2["response"].response_ms

In [ ]:
def func1(a: str, b: int, c: int = 1, d: Optional[List[Dict]] = None) -> str: 
    """
    This is the short description.

    Here we may have a longer description. This one can go over many lines

    :param str a: this is parameter a
    :param b: this is parameter b
    :type b: int
    :param c: some parameter c    
    :param d: some parameter d
    :return: what it returns
    :rtype: str
    """
    return "x"


In [ ]:
llms.make_tooling(func1)

In [ ]:
doc = docstring_parser.parse(func1.__doc__)

In [ ]:
doc.params[0]

In [ ]:
len(doc.params)

## Test Retries

In [ ]:
config2 = dict(
    llms = [
        dict(llm="ollama/llama3", api_url="http://localhost:11434", num_retries=3)
    ]
)
config2

In [ ]:
llms2 = LLMS(config2, use_phoenix=("http://0.0.0.0:6006/v1/traces", "llms_wrapper_test"))
# llms2 = LLMS(config2)
llms2["ollama/llama3"].config

In [ ]:
messages = llms2.make_messages(query="What is a monoid")

In [ ]:
llms["openai/gpt-4o"].config

In [ ]:
ret = llms.query(
    "openai/gpt-4o", 
    messages=messages, 
    # return_cost=True,
    debug=True,
    num_retries=0,
)
ret